In [40]:
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
import numpy as np
import random
import plotly.express as px
from plotly.offline import iplot
from tqdm import tqdm

In [41]:
cancer_set = load_breast_cancer()

[data sepecification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)

# 로지스틱 회귀
로지스텍 모델(logistic model), 로짓 모델(logit model)은 로그-오즈(log-odds) 사건을 선형조합을 통해 예측하는 것을 의미합니다. 로지스틱 회귀(logistic regression)는 이에 대한 매개변수를 예측하기 위한 알고리즘입니다.
이진 회귀 분석(binary logistic regression)에선 지시변수(indicator variable)를 독립변수로 해독해 사용합니다—이때, 지시변수는 0과 1로 주어지며 독립변수는 결국 0 ~ 1의 연속변수(continuous variable)가 됩니다.
- 두 개 이상의 이진변수를 독립변수로 사용하는 경우엔 다중 회귀 분석(multinomial logistic regression)라 합니다. 이때, 다수의 이진변수는 곧 범주형 변수(categorical variable)로 다시 정형화될 수 있습니다—이때, 각 범주들이 정렬되었고 이들의 연속적 특징이 의미를 가질 때, 정렬적 로지스틱 회귀(ordinal logistic regression)라 합니다.
- 로지스틱 회귀 자체는 단순한 확률적 예측을 제공할 뿐이지만 분류기로써도 사용될 수 있으며 이러한 형태를 통계적 분류기(statical classifier)라고 합니다.

In [42]:
class Dataset:
  def __init__(self, dataset, indices, transform=None, encoder=None):
    self.dataset, self.indices = dataset, indices
    self.transform, self.encoder = transform, encoder
  def __getitem__(self, item: int):
    idx = self.indices[item]
    feature, label = self.dataset.data[idx], self.dataset.target[idx]
    if self.transform: feature = self.transform(feature)
    if self.encoder: label = self.encoder(label)
    return feature, label
  def __len__(self): return len(self.indices)

In [43]:
indices = random.sample(range(cancer_set.data.__len__()), 100)

# init Datasets
support_set = Dataset(cancer_set, indices[:50])
query_set = Dataset(cancer_set, indices[50:])

## 손실함수 정의하기
손실함수(loss function)는 기계학습에서 모델의 예측값과 실제값 사이의 차이를 측정하는 함수입니다. 손실 함수는 모델이 얼마나 잘 수행되고 있는지를 수치적으로 나타내며, 이 값을 최소화하는 것이 모델 학습의 목표입니다.
* $\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
* $\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$
> 전체적 관찰에 대한 오차를 위한 함수는 비용함수(cost function)라고 별칭합니다.

In [50]:
# define loss functions(MSE, MAE)
def mean_squared_error(independent, dependent, weight):
  probability = np.dot(independent, weight)
  return np.mean((probability - dependent) ** 2)
# mean_squared_error

def mean_absolute_error(independent, dependent, weight):
  probability = np.dot(independent, weight)
  return np.mean(abs(probability - dependent))
# mean_absolute_error

In [51]:
def sigmoid(x): return 1 / (1 + np.exp(-1 * np.clip(x, -1e2, 1e2)))

## 수식화
$\left\{ y_i, x_{i1}, …, x_{ip} \right\}^n_{i=1}$를 통해 선형모델은 독립변수 y와 종속변수 $\beta$를 예측해야 합니다.
$$y_{i} = \beta_{0} + \beta_{1}x_{i1} + ... + \beta_{p}x_{ip} + \epsilon_{i} = \beta_{p}x_{ip} + \epsilon_{i}$$
* $y$는 관측값의 벡터입니다.
* $x$는 열 벡터 $x_i$의 행렬 또는 다차원의 행 벡터 $x_j$입니다.
* $\beta$는 $p+1$ 차원의 매개변수 벡터입니다.
* $\epsilon$는 $\epsilon_i$의 벡터입니다.

In [53]:
class LogisticRegression:
  def __init__(self, n_inpt): self.weight = np.zeros(shape=(n_inpt))
  def gdr(self, x, y, lr):
    indications = self.forward(x)
    self.weight -= (lr / x.shape[0]) * np.dot(x.T, (indications - y))
  # gdr
  def train(self, dataset, iters: int, lr=0.01):
    for _ in range(iters):
      for feature, label in dataset: self.gdr(feature, label, lr=lr)
  # train
  def forward(self, x): return sigmoid(np.dot(x, self.weight))
# LogisticRegression

## 경사 하강법
경사하강법(Gradient Descent)은 머신러닝과 최적화 분야에서 널리 사용되는 알고리즘으로, 함수의 최솟값을 찾기 위해 반복적으로 파라미터를 업데이트하는 방법입니다. 주로 모델의 손실을 최소화하는 데 사용되며, 이 과정에서 모델의 가중치를 조정합니다.
1. 가중치 $\theta$를 임의의 값을 초기화합니다.
2. 현재의 가중치 $\theta$에서 손실함수의 경사 $\nabla{f(\theta)}$를 계산합니다.
3. 가중치 갱신하기: $\theta_{\text{new}} \leftarrow \theta - \mathcal{n}\cdot\nabla{f(\theta)}$
4. 경사가 충분히 작아질 때까지 2 ~ 3을 반복합니다.

In [54]:
def GDR(model, lr):
  def _GDR(x, y):
    pred = model.forward(x)
    model.weight -= lr * np.dot(x.T, (pred - y))
  return _GDR

In [55]:
progress_bar = tqdm(range(100))

# init and train a model
model = LogisticRegression(30)
optimizer = GDR(model, 0.001)
for _ in progress_bar:
  loss = 0.
  for feature, label in support_set:
    optimizer(feature, label)
    loss += mean_squared_error(feature, label, model.weight)
  progress_bar.set_postfix(loss=loss/len(support_set))

100%|██████████| 100/100 [00:00<00:00, 391.40it/s, loss=1.66e+6]


## 검증하기
아래 검증 결과를 통해 해당 구현이 `0.92`의 정확도를 달성했음을 알 수 있습니다.

In [56]:
count, n_samples = 0, len(query_set)
for feature, label in support_set:
  pred = model.forward(feature)
  if round(pred) == label: count += 1
print(f"accuracy: {count / n_samples:.2f}({count}/{n_samples})")

accuracy: 0.94(47/50)
